# Setup

Import libraries

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from datetime import datetime
import numpy as np
import pandas as pd

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
from torch import nn

from keras.preprocessing.sequence import pad_sequences

from tqdm.notebook import tqdm
from sklearn import metrics

from tape import ProteinBertModel, TAPETokenizer
import matplotlib.pyplot as plt

random_seed = 22

_ = torch.manual_seed(random_seed)

Define paths and experiment name

In [ ]:
project_dir = "/path/to/project/root"

data_dir = project_dir + "/data"
models_dir = data_dir + "/models"
temp_dir = project_dir + "/temp"
datasets_dir = data_dir + "/datasets"

hla_seqs_file = data_dir + "/carmen-hla-sequences.parquet"

time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S-%f")

temp_output_no_hla_file = temp_dir + f"/test_prediction_no_hla_{time_stamp}"
temp_output_hla_file = temp_dir + f"/test_prediction_with_hla_{time_stamp}"

temp_output_no_hla_flat_file = temp_dir + f"/test_prediction_no_hla_flat_{time_stamp}"
temp_output_hla_flat_file = temp_dir + f"/test_prediction_with_hla_flat_{time_stamp}"

if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

dataset_src_col_pep = "peptide"
dataset_src_col_label = "label"
dataset_src_col_hla_seq = "Sequence"

dataset_true_col_pep = "peptide"
dataset_true_col_hla_seq = "mhc"
dataset_true_col_label = "label"

dataset_decoys_col_pep = "peptide_rand"
dataset_decoys_col_hla_seq = "mhc_rand"
dataset_decoys_col_label = "label"

model_type = "tape"

Select experiment type

In [ ]:
exp_name = "carmen_balanced_1_1"

# exp_name = "carmen_balanced_1_5"

# exp_name = "carmen_unbalanced_1_1"

# exp_name = "carmen_unbalanced_1_5"

# exp_name = "synthetic_easy"

# exp_name = "synthetic_hard"

In [ ]:
weights_no_hla_file = models_dir + f"/model_{model_type}_{exp_name}_no_hla.pt"
weights_hla_file = models_dir + f"/model_{model_type}_{exp_name}_with_hla.pt"

test_set_dir = datasets_dir + f"/{exp_name}"

carmen_test_file = test_set_dir + "/test_set.csv"

synthetic_true_test_file = test_set_dir + "/true_dataset_test.csv"
synthetic_decoys_test_file = test_set_dir + "/decoys_dataset_test.csv"

Import the tokenizer

In [ ]:
tokenizer = TAPETokenizer(vocab="iupac")

Define the model

In [ ]:
class TAPEClassification(nn.Module):
    def __init__(self, input_dim_bert, output_dim):
        super().__init__()
        self.bert = ProteinBertModel.from_pretrained("bert-base")
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(input_dim_bert, output_dim)

    def forward(self, x_sem):
        pooled_output = self.bert(x_sem)[0][:, 0, :]
        pooled_output = self.dropout(pooled_output)
        out = self.classifier(pooled_output)

        return out

In [ ]:
model = TAPEClassification(768, 2)

model.load_state_dict(torch.load(weights_no_hla_file))

model.cuda()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# STEP 1: No HLA

In [ ]:
if exp_name.startswith("carmen_"):
    df_test = pd.read_csv(carmen_test_file)

    test_peptides = df_test.peptide.values
    test_labels_no_hla = df_test.label.values
elif exp_name.startswith("synthetic_"):
    dataset_true_test = pd.read_csv(synthetic_true_test_file, index_col=0)
    dataset_decoys_test = pd.read_csv(synthetic_decoys_test_file, index_col=0)

    dataset_true_test = dataset_true_test.rename(
        columns={dataset_true_col_hla_seq: dataset_src_col_hla_seq}
    )
    dataset_decoys_test = dataset_decoys_test[
        [dataset_decoys_col_pep, dataset_decoys_col_label, dataset_decoys_col_hla_seq]
    ].rename(
        columns={
            dataset_decoys_col_pep: dataset_src_col_pep,
            dataset_decoys_col_hla_seq: dataset_src_col_hla_seq,
        }
    )

    df_test = pd.concat(
        [dataset_true_test, dataset_decoys_test], ignore_index=True, axis=0
    )

    del dataset_true_test, dataset_decoys_test

    df_test[dataset_src_col_label] = df_test[dataset_src_col_label].replace(
        {True: 1, False: 0}
    )

    test_peptides = df_test.peptide.values
    test_labels_no_hla = df_test.label.values

del df_test

Tokenize the input

In [ ]:
print("Tokenizing inputs")

tokenized_test_peptide = [tokenizer.encode(sent) for sent in test_peptides]
tokenized_test = pad_sequences(
    [sent for sent in tokenized_test_peptide],
    dtype="long",
    truncating="post",
    padding="post",
)

test_inputs = torch.tensor(tokenized_test)
test_labels = torch.tensor(test_labels_no_hla)

test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=32)

Run the model

In [ ]:
with open(temp_output_no_hla_file, "w") as f:
    pass
with open(temp_output_no_hla_flat_file, "w") as f:
    pass

model.eval()

for batch in tqdm(test_dataloader, desc="Testing 'no HLA' model"):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_labels = batch

    with torch.no_grad():
        logits = model(b_input_ids)
    logits = logits.detach().cpu().numpy()

    with open(temp_output_no_hla_file, "a") as f:
        for elem in logits[:, 1]:
            f.write(str(elem) + " ")

    flat_predictions = np.argmax(logits, axis=1).flatten()
    with open(temp_output_no_hla_flat_file, "a") as f:
        for elem in flat_predictions:
            f.write(str(elem) + " ")

# STEP 2: With HLA

Import the HLA sequence file

In [ ]:
if exp_name.startswith("carmen_"):
    df_test_HLA = pd.read_csv(carmen_test_file)
    HLA_df = pd.read_parquet(hla_seqs_file)
    HLA_df = HLA_df.rename(columns={"Allele": "HLA"})
    df_test_HLA = df_test_HLA.merge(HLA_df, on="HLA")
    df_test_HLA = df_test_HLA.drop(columns=["Pseudo_sequence"])

    test_peptides = df_test_HLA.peptide.values
    HLA_sequences = df_test_HLA.Sequence.values
    test_labels_with_hla = df_test_HLA.label.values
elif exp_name.startswith("synthetic_"):
    dataset_true_test = pd.read_csv(synthetic_true_test_file, index_col=0)
    dataset_decoys_test = pd.read_csv(synthetic_decoys_test_file, index_col=0)

    dataset_true_test = dataset_true_test.rename(
        columns={dataset_true_col_hla_seq: dataset_src_col_hla_seq}
    )
    dataset_decoys_test = dataset_decoys_test[
        [dataset_decoys_col_pep, dataset_decoys_col_label, dataset_decoys_col_hla_seq]
    ].rename(
        columns={
            dataset_decoys_col_pep: dataset_src_col_pep,
            dataset_decoys_col_hla_seq: dataset_src_col_hla_seq,
        }
    )

    df_test_HLA = pd.concat(
        [dataset_true_test, dataset_decoys_test], ignore_index=True, axis=0
    )

    del dataset_true_test, dataset_decoys_test

    df_test_HLA[dataset_src_col_label] = df_test_HLA[dataset_src_col_label].replace(
        {True: 1, False: 0}
    )

    test_peptides = df_test_HLA.peptide.values
    HLA_sequences = df_test_HLA.Sequence.values
    test_labels_with_hla = df_test_HLA.label.values

del df_test_HLA

Tokenize and concat the input

In [ ]:
def tokenize_input(peptide_sequences, HLA_sequences):
    tokenized_sequence = []

    for index in tqdm(range(0, len(peptide_sequences)), desc="Tokenizing inputs"):
        peptide = peptide_sequences[index]
        HLA = HLA_sequences[index]

        tokenized_peptide = [tokenizer.encode(peptide)]
        tokenized_HLA = [tokenizer.encode(HLA)]

        result = np.concatenate((tokenized_peptide, tokenized_HLA[0][1:]), axis=None)

        tokenized_sequence.append(result)

    return tokenized_sequence

In [ ]:
tokenized_peptide = tokenize_input(test_peptides, HLA_sequences)

tokenized_test = pad_sequences(
    [sent for sent in tokenized_peptide],
    dtype="long",
    truncating="post",
    padding="post",
)

test_inputs = torch.tensor(tokenized_test)
test_labels = torch.tensor(test_labels_with_hla)

test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=32)

Load and run the model

In [ ]:
model.load_state_dict(torch.load(weights_hla_file))

model.cuda()

In [ ]:
with open(temp_output_hla_file, "w") as f:
    pass
with open(temp_output_hla_flat_file, "w") as f:
    pass

model.eval()

for batch in tqdm(test_dataloader, desc="Testing 'with HLA' model"):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_labels = batch

    with torch.no_grad():
        logits = model(b_input_ids)
    logits = logits.detach().cpu().numpy()

    with open(temp_output_hla_file, "a") as f:
        for elem in logits[:, 1]:
            f.write(str(elem) + " ")

    flat_predictions = np.argmax(logits, axis=1).flatten()
    with open(temp_output_hla_flat_file, "a") as f:
        for elem in flat_predictions:
            f.write(str(elem) + " ")

Read output files

In [ ]:
with open(temp_output_no_hla_flat_file, "r") as f:
    data = f.read()
data_list = data.split(" ")
array_list = np.array(data_list[:-1])
no_HLA_predictions_f = array_list.astype(int)
os.remove(temp_output_no_hla_flat_file)

with open(temp_output_hla_flat_file, "r") as f:
    data = f.read()
data_list = data.split(" ")
array_list = np.array(data_list[:-1])
HLA_predictions_f = array_list.astype(int)
os.remove(temp_output_hla_flat_file)

print("No HLA:")
acc_no_hla = metrics.accuracy_score(test_labels_no_hla, no_HLA_predictions_f)
print(f"Accuracy: {acc_no_hla}")
f1_no_hla = metrics.f1_score(test_labels_no_hla, no_HLA_predictions_f)
print(f"F1: {f1_no_hla}")
prec_no_hla = metrics.precision_score(test_labels_no_hla, no_HLA_predictions_f)
print(f"Precision: {prec_no_hla}")
rec_no_hla = metrics.recall_score(test_labels_no_hla, no_HLA_predictions_f)
print(f"Recall: {rec_no_hla}")
print()
print("With HLA:")
acc_with_hla = metrics.accuracy_score(test_labels_with_hla, HLA_predictions_f)
print(f"Accuracy: {acc_with_hla}")
f1_with_hla = metrics.f1_score(test_labels_with_hla, HLA_predictions_f)
print(f"F1: {f1_with_hla}")
prec_with_hla = metrics.precision_score(test_labels_with_hla, HLA_predictions_f)
print(f"Precision: {prec_with_hla}")
rec_with_hla = metrics.recall_score(test_labels_with_hla, HLA_predictions_f)
print(f"Recall: {rec_with_hla}")

In [ ]:
with open(temp_output_no_hla_file, "r") as f:
    data = f.read()

data_list = data.split(" ")

array_list = np.array(data_list[:-1])
no_HLA_predictions = array_list.astype(float)

os.remove(temp_output_no_hla_file)

In [ ]:
with open(temp_output_hla_file, "r") as f:
    data = f.read()

data_list = data.split(" ")

array_list = np.array(data_list[:-1])
HLA_predictions = array_list.astype(float)

os.remove(temp_output_hla_file)

# Plot

In [ ]:
fpr_no_HLA, tpr_no_HLA, _ = metrics.roc_curve(test_labels_no_hla, no_HLA_predictions)
auc_no_HLA = metrics.roc_auc_score(test_labels_no_hla, no_HLA_predictions)

In [ ]:
fpr_HLA, tpr_HLA, _ = metrics.roc_curve(test_labels_with_hla, HLA_predictions)
auc_HLA = metrics.roc_auc_score(test_labels_with_hla, HLA_predictions)

In [ ]:
auc_round = 5

auc_no_HLA = round(auc_no_HLA, auc_round)
auc_HLA = round(auc_HLA, auc_round)

In [ ]:
fig = plt.figure(figsize=(12, 8))

plt.plot(
    fpr_no_HLA,
    tpr_no_HLA,
    label="Without HLA (AUC=" + str(auc_no_HLA) + ")",
    linewidth=2,
)
plt.plot(fpr_HLA, tpr_HLA, label="With HLA (AUC=" + str(auc_HLA) + ")", linewidth=2)

plt.ylabel("True positive rate", fontsize=24)
plt.xlabel("False positive rate", fontsize=24)
plt.title(f"ROC curve comparison: {exp_name}", fontsize=24, pad=20)

plt.grid(linestyle="--")

_ = plt.legend(fontsize=20)

In [ ]:
fig = plt.figure(figsize=(12, 8))

w = 0.4

plt.bar(
    [0 - w / 2, 1 - w / 2, 2 - w / 2, 3 - w / 2],
    [acc_no_hla, f1_no_hla, prec_no_hla, rec_no_hla],
    color="C0",
    label="Without HLA",
    width=w,
)

plt.bar(
    [0 + w / 2, 1 + w / 2, 2 + w / 2, 3 + w / 2],
    [acc_with_hla, f1_with_hla, prec_with_hla, rec_with_hla],
    color="C1",
    label="With HLA",
    width=w,
)

plt.xticks([0, 1, 2, 3], ["Accuracy", "F1", "Precision", "Recall"], fontsize=12)

_ = plt.legend(fontsize=15)